# 概要

このノートブックでは、YouTube Data API v3を使用してプレイリストから動画情報やコメントから楽曲一覧を取得できます。

## 実行できるスクリプト

1. **fetch_comments.py**：YouTubeの再生リストIDから各動画のコメント欄を取得し、投稿されたタイムスタンプを元に楽曲のリストを作成しCSVファイル形式でダウンロードします
2. **fetch_videos.py**：YouTubeの再生リストIDから各種動画IDと動画タイトルを抽出しJSONファイルとしてダウンロードします


# APIキー（token）の取得方法

各PythonスクリプトではYouTube Data API v3を使用するため、対応するAPIキーが必要になります。

## 取得手順

1. [Google Cloud Console](https://console.cloud.google.com/)にアクセス
2. 新しいプロジェクトを作成（または既存のプロジェクトを選択）
3. 「APIとサービス」→「ライブラリ」から「YouTube Data API v3」を有効化
4. 「認証情報」→「認証情報を作成」→「APIキー」を選択（制限は不要）
5. 作成されたAPIキーをコピー

**注意**: APIキーは安全に管理してください。公開リポジトリにコミットしないよう注意してください。

# セットアップ

## 1. 必要なパッケージのインストール

以下のコマンドで必要なPythonパッケージをインストールします。

In [ ]:
%pip install requests

## 2. スクリプトの取得

GitHubリポジトリからスクリプトをダウンロードします。

In [ ]:
!wget https://raw.githubusercontent.com/LiFE-124C41/uta_picker/main/script/fetch_comments.py
!wget https://raw.githubusercontent.com/LiFE-124C41/uta_picker/main/script/fetch_videos.py

## 3. APIキー（token）の入力

YouTube Data API v3のAPIキーを入力します。入力した値は表示されません（セキュリティのため）。

**注意**: このセルで入力したAPIキーは、このノートブック内の両方のスクリプトで使用されます。

In [ ]:
import getpass
token = getpass.getpass("Enter API key: ")

# fetch_comments.py実行

## 再生リストIDの指定

再生リストIDは、YouTubeプレイリストのURLの`list=`パラメータの値です。

**例**: `https://www.youtube.com/playlist?list=PLxxxxxxx` の場合、`PLxxxxxxx`が再生リストIDです。

プレイリストIDは通常`PL`で始まる文字列です。

In [ ]:
list_id_comments = "PLw6tV8gK-20tVH2w-ZK-SAZ5lErzMLrj5" # @param {type:"string"}

## スクリプト実行

以下のコマンドでスクリプトを実行します。処理には時間がかかる場合があります。

In [ ]:
import subprocess
result = subprocess.run(['python', 'fetch_comments.py', token, list_id_comments], 
                       capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("エラー:", result.stderr)

## 出力ファイルの確認

スクリプト実行後、`comments.csv`ファイルが作成されます。このファイルには以下の情報が含まれます：

- `video_title`: 動画のタイトル
- `song_title`: 抽出された曲名（タイムスタンプの前後から自動抽出）
- `video_id`: YouTube動画ID
- `start_sec`: 曲の開始時間（秒）
- `end_sec`: 曲の終了時間（秒、次のタイムスタンプまたは開始時間+300秒）
- `link`: 動画の該当時間へのリンク

ファイルをダウンロードするには、左側のファイル一覧から`comments.csv`を右クリックして「ダウンロード」を選択してください。

In [ ]:
# ファイルが存在するか確認
import os
if os.path.exists('comments.csv'):
    print("✓ comments.csv が正常に作成されました")
    # ファイルの最初の数行を表示
    with open('comments.csv', 'r', encoding='utf-8') as f:
        lines = f.readlines()[:5]
        print("\nファイルの最初の数行:")
        print(''.join(lines))
else:
    print("⚠ comments.csv が見つかりません。エラーが発生した可能性があります。")

# fetch_videos.py実行

## 再生リストIDの指定

再生リストIDは、YouTubeプレイリストのURLの`list=`パラメータの値です。

**例**: `https://www.youtube.com/playlist?list=PLxxxxxxx` の場合、`PLxxxxxxx`が再生リストIDです。

プレイリストIDは通常`PL`で始まる文字列です。

In [ ]:
list_id_videos = "PLw6tV8gK-20tVH2w-ZK-SAZ5lErzMLrj5" # @param {type:"string"}

## スクリプト実行

以下のコマンドでスクリプトを実行します。

In [ ]:
import subprocess
result = subprocess.run(['python', 'fetch_videos.py', token, list_id_videos], 
                       capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("エラー:", result.stderr)

## 出力ファイルの確認

スクリプト実行後、`videos.json`ファイルが作成されます。このファイルには以下の情報が含まれます：

- `videoId`: YouTube動画ID
- `title`: 動画のタイトル
- `publishedAt`: 動画の公開日時

ファイルをダウンロードするには、左側のファイル一覧から`videos.json`を右クリックして「ダウンロード」を選択してください。


In [ ]:
# ファイルが存在するか確認
import os
import json
if os.path.exists('videos.json'):
    print("✓ videos.json が正常に作成されました")
    # ファイルの内容を読み込んで表示
    with open('videos.json', 'r', encoding='utf-8') as f:
        videos = json.load(f)
        print(f"\n取得した動画数: {len(videos)}件")
        if videos:
            print("\n最初の動画情報:")
            print(json.dumps(videos[0], ensure_ascii=False, indent=2))
else:
    print("⚠ videos.json が見つかりません。エラーが発生した可能性があります。")


# 注意事項

## 一般的な注意事項

- **APIキーの管理**: APIキーは安全に管理してください。公開リポジトリにコミットしないよう注意してください
- **APIレート制限**: YouTube Data APIには1日のリクエスト数に制限があります（デフォルトで10,000ユニット/日）
- **処理時間**: コメント取得には時間がかかる場合があります（APIレート制限を避けるため、各動画の処理後に0.1秒待機）

## fetch_comments.py に関する注意事項

- タイムスタンプが見つからないコメントはスキップされます
- 曲名が抽出できない場合は、`曲@タイムスタンプ`の形式で記録されます
- 最後のエントリの終了時間は、開始時間+300秒（5分）がデフォルトとして設定されます
- プレイリストに含まれる動画数が多い場合、処理に時間がかかります

## fetch_videos.py に関する注意事項

- デフォルトでは最大50件の動画を取得します
- プレイリストに含まれる動画数が50件を超える場合、最初の50件のみが取得されます

## エラーが発生した場合

- APIキーが正しいか確認してください
- 再生リストIDが正しいか確認してください（`PL`で始まる文字列である必要があります）
- プレイリストが公開されているか確認してください
- APIレート制限に達していないか確認してください
